# The Main Contracting Client
Contracting comes with a robust client that abstracts most of the underpinnings of the smart contracting system so that it feels like regular Python. This lets you worry about your programming rather than the mechanics of the protocol. To start, create an instance of the `ContractingClient` like so. You can set the default `signer` who will then automatically be the `signer` of the subsequent function calls (or transactions, in blockchain terms).

You can change the signer by setting this property on the client, or by overriding the keyword `sender` when calling a function.

In [1]:
from contracting.client import ContractingClient
client = ContractingClient(signer='stu')

## Viewing contracts

The client preloads a set of 'genesis' contracts that are required for the system to operate. These genesis contracts bypass some of the system protections to allow a channel between the low level protocol and the higher level programming interface to occur without exposing everything to the developer. This allows us to create powerful system level contracts while maintaining tight security on user level contracts.

One of these genesis contracts is the contract submission contract. Ironically, we have a smart contract to submit smart contracts. This is for purposes of conflict resolution, which is a later topic. For now, you can see the contracts by calling the function below.

In [2]:
client.get_contracts()

['submission']

## Interacting with existing smart contracts

Contracts can be fetched into their own objects with the `get_contract` method.

In [3]:
client.get_contract('submission')

You can see all available functions you can call on this smart contract by accessing the `self.functions` property. This returns a list of tuples made up of function names and the list of keyword arguments they require.

In [4]:
submission = client.get_contract('submission')
submission.functions

[('submit_contract', ['name', 'code'])]

## Submitting new smart contracts

Let's use the `submit_contract` function to submit a new smart contract!

In [5]:
code = '''
@export
def hello_there():
    print('howdy')
'''

name = 'howdy_time'

submission.submit_contract(name=name, code=code)

Now we can check to make sure that the smart contract made it into the `submission` contract's "namespace." Each smart contract has its own namespace where it stores data which isolates it from other smart contracts securely.

In [6]:
howdy_time = client.get_contract(name)

In [7]:
howdy_time.keys()

['howdy_time.__author__',
 'howdy_time.__code__',
 'howdy_time.__compiled__',
 'howdy_time.__type__']

In [8]:
print(howdy_time.__code__)

def hello_there():
    print('howdy')



It worked! But why was the @export decorator deleted? This actually is a lower level Contracting function that parses submitted code and transforms it into proper Python. In this process, it also checks to make sure that the code contains all of the correct formatting and features. This is what a contract looks like if you include a private function as well.

In [9]:
code = '''
@export
def secret():
    return abcd()
    
def abcd():
    return 5
'''

name = 'secret'

submission.submit_contract(name=name, code=code)

In [10]:
secret = client.get_contract('secret')

In [11]:
secret.functions

[('secret', []), ('__abcd', [])]

In [12]:
print(secret.__code__)

def secret():
    return __abcd()


def __abcd():
    return 5



In [13]:
secret.secret()

5

Notice how methods without @export on them get prepended with `__`. This prevents people from calling these methods in their own smart contracts, making them effectively private. The Contracting client, however, is designed for the maximal developer experience, so you may be able to call these functions from the client so that you can write unit tests and integration tests around their functionality.

## Closures and direct submission

Instead of pulling out the `submission` contract every time before a session, you can access it via the client on the `submit` method. This method also allows you to submit closure functions (functions that have other functions inside of them). As long as the code inside of the closure can be considered a valid smart contract, the client will convert it into a code string and submit it to the `submission` smart contract.

Contracts submitted this way will be named whatever their closure function is.

In [14]:
def closure_example():
    @export
    def hello():
        return 'How are you?'
    
    def shh():
        return 10

In [15]:
client.submit(closure_example)

In [18]:
closure = client.get_contract('closure_example')

In [20]:
print(closure.__code__)

def hello():
    return 'How are you?'


def __shh():
    return 10



Now that you have an idea on how to use the client, let's dive into the mechanics behind smart contracts in the next notebook.